CPN для XOR

In [ ]:
import math
import numpy as np

class CPN:
    def __init__(self, x, y, k_neurons, r_neurons):
        self.x = x
        self.y = y
        self.kohonen_weights = self.weights(k_neurons, len(x[0]))
        self.grossberg_weights = self.weights(r_neurons, len(x[0]))

    def weights(self, num_neurons, length): #Случайная инициализация весов
        result = np.asarray(np.random.rand(num_neurons, length))
        if len(result) == 1:
            return result[0]
        return result

    def euclid(self, w_vector, x_vector): #Евклидово расстояние
        return sum([((w - x) ** 2) for w, x in zip(w_vector, x_vector)])

    def sum_out(self, k_vector, w_vector): #Сумма выхода Кохонена на веса Гроссберга
        return sum([k * w for k, w in zip(k_vector, w_vector)])

    def new_k_weights(self, x_vector, w_vector, learning_rate=0.7): #Обновление весов слоя Кохонена
        weights = []
        for x, w in zip(x_vector, w_vector):
            w_new = w + learning_rate * (x - w)
            weights.append(w_new)
        return np.asarray(weights)

    def new_g_weights(self, y_value, w_value, learning_rate=0.1, k=1): #Обновление весов слоя Гроссберга
        w_new = w_value + learning_rate * (y_value - w_value) * k
        return w_new

    def teach(self, lr_k, lr_g, epochs):
        for epoch in range(epochs):
            print(f'epoch {epoch}')
            matching = 0 #Количество совпадений
            for x_vector, y_value in zip(self.x, self.y):
                k_neurons = []
                for w_iter in range(len(self.kohonen_weights)):
                    k_neurons.append(self.euclid(x_vector, self.kohonen_weights[w_iter])) #Расстояние между вектором и весами
                min_dist = min(k_neurons) #Минимальное расстояние
                index = k_neurons.index(min_dist)
                for i in range(len(k_neurons)): #Наименьшая дистанция - 1, остальные 0
                    if i == index:
                        k_neurons[i] = 1
                    else:
                        k_neurons[i] = 0
                self.kohonen_weights[index] = self.new_k_weights(x_vector, self.kohonen_weights[index], learning_rate=lr_k) #Пересчитываем веса
                self.grossberg_weights[index] = self.new_g_weights(y_value, self.grossberg_weights[index], learning_rate=lr_g) #Пересчитываем веса
                r_neuron_out = int(round(self.sum_out(k_neurons, self.grossberg_weights)))
                if y_value == r_neuron_out:
                  matching += 1
                print(f'О: {y_value}')
                print(f'П: {r_neuron_out}')
            print(f'Процент совпадений: {int(matching / len(self.y) * 100)}%')
            if int(matching / len(self.y) * 100) == 100: #Процент совпадений
              break

x = [[0,0], [0,1], [1,0],[1,1]]
y = [0, 1, 1, 0]
print('О - ожидаемое значение')
print('П - полученное значение')
net = CPN(x, y, k_neurons=2, r_neurons=1)
net.teach(lr_k=0.6, lr_g=0.2, epochs=20)

О - ожидаемое значение
П - полученное значение
epoch 0
О: 0
П: 0
О: 1
П: 0
О: 1
П: 0
О: 0
П: 0
Процент совпадений: 50%
epoch 1
О: 0
П: 0
О: 1
П: 0
О: 1
П: 0
О: 0
П: 0
Процент совпадений: 50%
epoch 2
О: 0
П: 0
О: 1
П: 1
О: 1
П: 1
О: 0
П: 0
Процент совпадений: 100%
